# Imports

In [187]:
import traveltimepy as ttpy
import os
from datetime import datetime

# Authentication

In [188]:
os.environ["TRAVELTIME_ID"] = '9e92dc47'
os.environ["TRAVELTIME_KEY"] = 'ea73781a50a84f6135b7a46ec5b3d03a'

# Constants

In [189]:
initial_travel_time = 60

In [190]:
source_locations = [
    "UB3 1FG",
    "Russell Road, Wimbledon",
    "Kilburn High Road"
]

# Requests

In [191]:
def location_coords(source_locations):
    geoencoded_locations = []
    for item in source_locations:
        geoencoded_locations.append(ttpy.geocoding(item)['features'][0]['geometry']['coordinates'])
    return geoencoded_locations

In [192]:
def gen_search_data(travel_time, source_coords):
    searches = []
    search_ids = []
    for idx, loc in enumerate(source_coords):
        searches.append({
            'id': str(idx),
            'departure_time':  datetime.utcnow().isoformat(),
            'travel_time': travel_time * 60,
            'coords': {'lat': loc[1], 'lng': loc[0]},
            'transportation': {'type': "public_transport"},
        })
        search_ids.append(str(idx))
    return searches, search_ids

In [193]:
def generate_intersection(search_ids):
    return {
    'id': "intersection",
    'search_ids': search_ids
    }

In [194]:
def obtain_map_data(searches, intersection):
  return ttpy.time_map(departure_searches=searches, intersections=[intersection])

In [195]:
def find_intersection(initial_travel_time, source_locations):
    travel_time = initial_travel_time
    source_coords = location_coords(source_locations)
    searches, search_ids = gen_search_data(travel_time, source_coords)
    intersection = generate_intersection(search_ids)
    return obtain_map_data(searches, intersection)

In [196]:
out = find_intersection(initial_travel_time, source_locations)

# Extract intersection coords

In [197]:
intersect_coords = []
for shell in out['results']:
    if shell['search_id'] == "intersection":
        for shell in shell['shapes']:
            shell_coords = []
            for coord in shell['shell']:
                shell_coords.append((coord['lat'],coord['lng']))
                print("{},{}".format(coord['lat'],coord['lng']))
            intersect_coords.append(shell_coords)

51.50516,-0.07768463
51.506115,-0.08070846
51.50669,-0.08348378
51.50701,-0.08601517
51.5077,-0.09263168
51.50822,-0.094294645
51.508293,-0.09473883
51.508602,-0.097182535
51.508602,-0.11224016
51.508495,-0.113074064
51.507908,-0.114952035
51.505543,-0.118732
51.504402,-0.11963694
51.4997,-0.120889805
51.49925,-0.11999167
51.498802,-0.117836125
51.497902,-0.119273156
51.496555,-0.11711761
51.496555,-0.11424356
51.495655,-0.112806536
51.495655,-0.10993248
51.496555,-0.10849546
51.496555,-0.105621405
51.49431,-0.10202884
51.49341,-0.10346587
51.492508,-0.10202884
51.49161,-0.10346587
51.49116,-0.10274736
51.49116,-0.10131033
51.49206,-0.099873304
51.49206,-0.09843628
51.492958,-0.09699925
51.492958,-0.0941252
51.49431,-0.09196966
51.494755,-0.09268817
51.494755,-0.09556223
51.495205,-0.09628074
51.497,-0.09628074
51.49745,-0.09556223
51.49745,-0.0941252
51.498352,-0.09268817
51.498352,-0.09125115
51.49745,-0.08981413
51.497902,-0.08909561
51.4997,-0.08909561
51.50105,-0.08694007
51.50015

# Find the centre of each shell

In [198]:
def find_shell_centre(intersect_coords):
    sum_x = 0
    sum_y = 0
    total_points = len(intersect_coords)
    if total_points == 0:
        return (-1, -1)
    for point in intersect_coords:
        sum_x += point[0]
        sum_y += point[1]
    return (sum_x/total_points, sum_y/total_points)

In [199]:
shell_centres = []
for item in intersect_coords:
    shell_centres.append(find_shell_centre(item))

# Reverse the geo-encoding

In [200]:
def location_strings(coords):
    location_strings = []
    for item in coords:
        if item[0] != -1 and item[1] != -1:
            location_strings.append(ttpy.geocoding_reverse(item[0], item[1])['features'][0]['properties']['name'])
        else:
            print("Error - no intersection found")
    return location_strings

In [201]:
locations = location_strings(shell_centres)

In [202]:
locations

['Collinson Walk, Scovell Estate, Borough, London Borough of Southwark, London, Greater London, England, SE1 1PU, United Kingdom',
 'Lansdowne Road and Clarendon Road Garden, Lansdowne Road, Lancaster West Estate, Notting Hill, Royal Borough of Kensington and Chelsea, London, Greater London, England, W11 3LL, United Kingdom',
 'Emanuel School Boat Club, The Promenade, Grove Park, London Borough of Hounslow, London, Greater London, England, W4 2SH, United Kingdom',
 'No 35 Mackenzie Walk, 29-35, Mackenzie Walk, Canary Wharf, Isle of Dogs, London Borough of Tower Hamlets, London, Greater London, England, E14 4PH, United Kingdom',
 'Victoria Dock Road, Custom House, Canning Town, London Borough of Newham, London, Greater London, England, E16 3AS, United Kingdom',
 '153, College Road, Willesden Green, Harlesden, London Borough of Brent, London, Greater London, England, NW10 3PG, United Kingdom',
 'Hanger Court, Hanger Green, Park Royal, London Borough of Ealing, London, Greater London, Eng

# Testing